In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

import re
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# read the data
df=pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
# get first 5 rows
df.head()
print(df['sentiment'].unique())

In [ ]:
# Strip any leading/trailing whitespace and convert to lowercase if necessary
df['sentiment'] = df['sentiment'].str.strip().str.lower()

# Check unique values again
print(df['sentiment'].unique())

# Now apply the mapping
# df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

df["sentiment"]=df["sentiment"].apply(lambda x: 1 if x == 'positive' else 0)
df.head()

In [ ]:
def clean_data(text):
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^a-zA-Z]', ' ', text) 
    text=text.lower()
    text=text.split()
    text=[word for word in text if word not in stopwords.words('english')]
    text=" ".join(text)
    return text

In [ ]:
df['cleaned_review']=df['review'].apply(clean_data)

In [ ]:
# look if things are set properly
df[df['sentiment']==1]

In [ ]:

from sklearn.model_selection import train_test_split
X=df["cleaned_review"]
y=df["sentiment"]

train_x,test_x,train_y,test_y=train_test_split(X,y,test_size=0.2,random_state=52)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF (term frequency inverse document frequency)
# without doing this step you wont be able to train the model as 
# the training data is in text format it is needs to be converted into numerical

vectorizer=TfidfVectorizer(max_features=6000)
x_train_tfidf=vectorizer.fit_transform(train_x)
x_test_tfidf=vectorizer.fit_transform(test_x)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr=LogisticRegression()

In [ ]:
model=lr.fit(x_train_tfidf,train_y)

In [ ]:
y_pred=model.predict(x_test_tfidf)

In [ ]:
# check the accuracy of the model
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
acc=accuracy_score(test_y,y_pred)
pre_c=precision_score(test_y,y_pred)
f1_scr=f1_score(test_y,y_pred)
print(acc,pre_c,f1_scr)